In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
# create path and open browser window
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

# establish url
url = 'https://mars.nasa.gov/news/'

# visit site
browser.visit(url)

# grab page html
nasa_html = browser.html

# create soup object
soup = BeautifulSoup(nasa_html,'html.parser')

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [/Users/lbates/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [3]:
# find title for the latest one, which is the one in the first box
news_title = soup.find_all('div',class_='content_title')
news_title = news_title[1].text
news_title

'NASA Ingenuity Mars Helicopter Prepares for First Flight'

In [4]:
# pulling the text from the paragraph
news_p = soup.find_all('div', class_= 'article_teaser_body')
news_p = news_p[0].text
news_p

'Now uncocooned from its protective carbon-fiber shield, the helicopter is being readied for its next steps.  '

In [5]:
# establish url
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

# visit site
browser.visit(url)

# grab page html
jpl_html = browser.html

# create soup object
soup = BeautifulSoup(jpl_html,'html.parser')

In [7]:
#Find the image at the top and click 
target = 'a[class="group  cursor-pointer block"]'
browser.find_by_tag(target).click()

# grab page html
target_html = browser.html

# create soup object
soup = BeautifulSoup(target_html,'html.parser')

image = soup.find_all('a', class_='BaseButton text-contrast-none w-full mb-5 -primary -compact inline-block')
featured_image_url = image[0]['href']
featured_image_url

'https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24138.jpg'

In [10]:
# establish url
facts_url = 'https://space-facts.com/mars/'

# visit site
browser.visit(url)

#pull the table from the site
tables =pd.read_html('https://space-facts.com/mars/')
#pull the specific table for just the Mars data, it's the first table
mars_tables = tables[0]
#rename columns
mars_tables.columns = ['Fact','Value']
#convert dataframe back to html
mars_tables_html = mars_tables.to_html
mars_tables_html

<bound method DataFrame.to_html of                    Fact                          Value
0  Equatorial Diameter:                       6,792 km
1       Polar Diameter:                       6,752 km
2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
3                Moons:            2 (Phobos & Deimos)
4       Orbit Distance:       227,943,824 km (1.38 AU)
5         Orbit Period:           687 days (1.9 years)
6  Surface Temperature:                   -87 to -5 °C
7         First Record:              2nd millennium BC
8          Recorded By:           Egyptian astronomers>

In [ ]:
# establish url
hemi_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

# visit site
browser.visit(url)

# grab page html
hemi_html = browser.html

# create soup object
soup = BeautifulSoup(hemi_html,'html.parser')

In [ ]:
#Find the all the titles to use for the click function 
title_list = soup.find_all('div', class_='description')

hemisphere_image_urls = []
#loop through the title_list and pull title and imagine url
for t in title_list:
    title = t.h3.text
    browser.find_by_text(title).click()

    # grab page html
    title_html = browser.html

    # create soup object
    soup = BeautifulSoup(title_html,'html.parser')

    image = soup.find_all('div', class_='downloads')
    image_url = image[0].li.a['href']
    
    #create mini-dictionary
    mini_dictionary = {'title': title, 'img_url':image_url}
    hemisphere_image_urls.append(mini_dictionary)
    
    # click back button
    browser.back()
    
hemisphere_image_urls

In [ ]:
# create connection to mongo database
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# create new data base
db = client.mars_db

# drop the collection if it already exists
db.mars.drop()

# instert our list of mini dictionaries from scraping
db.mars.insert_many(mars_dictionary)